In [1]:
from tensorflow_docs.vis import embed
from tensorflow import keras
from tensorflow.keras import backend as K
from imutils import paths

import gzip
import pickle
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

from rl_with_videos.preprocessors.convnet import convnet_preprocessor

C:\Users\irohc\anaconda3\envs\drl_proj\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\irohc\anaconda3\envs\drl_proj\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\irohc\anaconda3\envs\drl_proj\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\irohc\anaconda3\envs\drl_proj\lib\site-packages\tensorf

In [2]:
POSITIVE_PATH = "C:/nyu/DRL/final_project/dataset/acrobot-975-1000.pkl"
NEGATIVE_PATH = "C:/nyu/DRL/final_project/dataset/acrobot-25-50.pkl"

MAX_SEQ_LENGTH = 30

def load_sequence(VIDEO_PATH):
    with gzip.open(VIDEO_PATH, 'rb') as f:
        latest_samples = pickle.load(f)

    key = list(latest_samples.keys())[0]
    num_samples = latest_samples[key].shape[0]
    for field_name, data in latest_samples.items():
        assert data.shape[0] == num_samples, data.shape
    
    return latest_samples

In [3]:
p_sample = load_sequence(POSITIVE_PATH)
n_sample = load_sequence(NEGATIVE_PATH)

In [4]:
p_size = p_sample['observations'].shape[0]
n_size = n_sample['observations'].shape[0]

In [5]:
def prepare_input(x_sample, x_size):
    input_x = []
    seq = []
    for i in range(0, x_size):
        seq.append(x_sample['observations'][i])
        if(x_sample['terminals'][i]):
            seq = np.array(seq)
            select_frame = np.linspace(0, seq.shape[0]-1, MAX_SEQ_LENGTH,endpoint=True,retstep=True,dtype=int)[0]
            input_x.append(seq[select_frame])
            seq = []
    return np.array(input_x)

def merge_input(p_input, n_input):
    input_batch = []
    label = []
    for i in range(p_input.shape[0]):
        input_batch.append(p_input[i])
        label.append([1])
    for i in range(n_input.shape[0]):
        input_batch.append(n_input[i])
        label.append([0])
    
    input_batch = np.array(input_batch).astype('float32')
    label = np.array(label)
    return input_batch, label

In [6]:
p_input = prepare_input(p_sample, p_size)
n_input = prepare_input(n_sample, n_size)

In [7]:
train_x, train_y = merge_input(p_input, n_input)

In [8]:
train_x.shape

(1287, 30, 6)

In [9]:
train_y.shape

(1287, 1)

In [10]:
save_dict = {}
save_dict['sequences'] = train_x
save_dict['done'] = train_y

In [11]:
output = open('acrobot_sequence.pkl', 'wb')

In [12]:
output.write(gzip.compress(pickle.dumps(save_dict)))

858802

In [13]:
pickle.dump(save_dict, output)

In [14]:
with gzip.open('./acrobot_sequence.pkl', 'rb') as f:
    latest_samples = pickle.load(f)

In [15]:
key = list(latest_samples.keys())[0]
num_samples = latest_samples[key].shape[0]

In [16]:
key

'sequences'

In [17]:
for field_name, data in latest_samples.items():
    print(data.shape)
    assert data.shape[0] == num_samples, data.shape

(1287, 30, 6)
(1287, 1)


In [18]:
values = latest_samples.get('done')

In [19]:
values

array([[1],
       [1],
       [1],
       ...,
       [0],
       [0],
       [0]])